In [26]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [27]:
# Import packages
from pyspark.sql import SparkSession
import time
from pyspark.sql import Row
from pyspark.sql.types import StructType,StructField,StructField,StructType,DateType,IntegerType

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [28]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

df = spark.read.option('header' , 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep= ',' ,timestampFormat="yyyy-MM-dd-HH.mm" )


In [30]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceGlobalTempView('home_sales')
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [31]:
from ast import alias
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
from pyspark.sql.functions import mean
from pyspark.sql.functions import round , col
four_bed_per_year = df.filter(df.bedrooms ==4)\
    .groupBy("date")\
    .agg(round(mean("price"), 2).alias("avg_price"))

four_bed_per_year.show()



+----------+---------+
|      date|avg_price|
+----------+---------+
|2021-11-13| 358445.0|
|2019-05-08| 234979.5|
|2021-08-27|275547.25|
|2021-01-27|367520.57|
|2021-06-22| 297708.8|
|2021-10-11|283388.33|
|2020-08-24|301080.75|
|2022-03-28|338544.78|
|2019-06-04|306373.08|
|2021-12-18| 305264.3|
|2020-11-29|318153.89|
|2020-01-21| 420576.0|
|2020-07-24| 293661.5|
|2021-11-25|291762.27|
|2019-11-18|290259.83|
|2019-09-22| 284864.5|
|2020-08-05|243835.56|
|2019-11-01|254751.38|
|2021-10-02|303838.38|
|2020-08-29| 285457.3|
+----------+---------+
only showing top 20 rows



In [32]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
three_bed_three_bath_per_year = df.filter((df.bedrooms == 3) & (df.bathrooms == 3)) \
 .groupBy("date") \
  .agg(round(mean("price"), 2).alias("avg_price"))


three_bed_three_bath_per_year.show()


+----------+---------+
|      date|avg_price|
+----------+---------+
|2019-05-08|285754.71|
|2021-06-22| 306565.0|
|2021-01-27|385711.75|
|2020-08-24|319252.33|
|2021-11-13|348180.33|
|2022-03-28| 280464.4|
|2021-10-11| 289886.4|
|2021-12-18| 178508.5|
|2021-08-27|298209.33|
|2019-06-04| 223109.0|
|2020-08-05|304566.75|
|2019-11-18|319833.67|
|2020-01-21| 303045.8|
|2021-11-25|321816.67|
|2019-11-01| 279354.0|
|2020-07-24| 283740.0|
|2020-11-29| 366673.0|
|2021-10-02| 400307.5|
|2019-09-22| 350352.5|
|2020-04-30| 192065.5|
+----------+---------+
only showing top 20 rows



In [33]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
avg_price_per_year = df.filter(
    (df.bedrooms == 3) &
    (df.bedrooms == 3) &
    (df.floors == 2) &
    (df.sqft_living >= 2000)
) \
     .groupBy("date") \
     .agg(round(mean("price"), 2).alias("avg_price"))
avg_price_per_year.show()


+----------+---------+
|      date|avg_price|
+----------+---------+
|2021-06-22| 420422.0|
|2021-08-27|341649.25|
|2019-05-08| 326690.5|
|2021-10-11| 245229.5|
|2021-01-27| 314084.0|
|2021-12-18| 300312.4|
|2020-08-24| 125408.0|
|2019-06-04| 165028.5|
|2021-11-13| 697736.0|
|2022-03-28| 204818.0|
|2019-11-01|244169.75|
|2021-10-02|250216.33|
|2019-11-18| 317416.5|
|2020-07-24| 185629.0|
|2020-08-05| 168276.0|
|2019-09-22| 283765.0|
|2020-01-21| 337179.5|
|2020-11-29| 232073.0|
|2020-04-30| 232217.0|
|2022-01-31| 284811.0|
+----------+---------+
only showing top 20 rows



In [34]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00012493133544921875 seconds ---


In [36]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("global_temp.home_sales")

In [37]:
# 8. Check if the table is cached.
spark.catalog.isCached("global_temp.home_sales")

True

In [38]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 6.890296936035156e-05 seconds ---


In [39]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').parquet('p_sales' , mode ='overwrite')

In [40]:
# 11. Read the parquet formatted data.
p_df = spark.read.parquet('p_sales')

In [41]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceGlobalTempView('p_sales')

In [42]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 8.463859558105469e-05 seconds ---


In [43]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("global_temp.home_sales")

In [44]:
# 15. Check if the home_sales is no longer cached
spark.stop()
